## Slack API 연결

In [1]:
import os, logging, requests
from pathlib import Path
from dotenv import load_dotenv

from slack_bolt import App
from slack_bolt.adapter.socket_mode import SocketModeHandler

#### 환경변수에서 API Key 불러오기

In [2]:
load_dotenv('../.env')
SLACK_BOT_TOKEN = os.getenv("SLACK-BOT")
SLACK_APP_TOKEN = os.getenv("SLACK-APP")

#### logging 설정

In [3]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

#### Slack Bolt Class 객체화

In [4]:
app = App(token=SLACK_BOT_TOKEN)

#### 현재 경로 저장

In [5]:
current_path = Path.cwd()

#### 파일 다운로드 함수

In [6]:
def download_file(file_url, file_name):   # 파일을 다운로드합니다
    response = requests.get(file_url, headers={"Authorization": f"Bearer {SLACK_BOT_TOKEN}"})
    if response.status_code == 200:
        # 파일을 저장합니다
        with open(file_name, "wb") as f:
            f.write(response.content)
        return True
    else:
        return False

In [7]:
# 모든 이벤트를 로깅하는 미들웨어
# Slack SDK(특히 Bolt 프레임워크)에서 **미들웨어(Middleware)**는 이벤트가 애플리케이션에서 처리되기 전에, 요청(request)을 전처리하거나, 애플리케이션의 동작을 확장하는 데 사용되는 코드 블록
# 미들웨어는 모든 이벤트나 요청이 애플리케이션의 특정 핸들러로 전달되기 전에 실행되므로, 공통 작업을 처리하는 데 유용함
# @app.middleware 데코레이터는 이 함수를 애플리케이션의 미들웨어로 등록
@app.middleware
def log_request(logger, body, next):
    print(f"이벤트 수신: {body['event']['type']}")
    next() # next()를 호출해 다음 단계로 처리를 넘김.

#### 파일 공유 이벤트 처리

In [8]:
# 파일이 공유될 때 실행되는 이벤트 리스너
@app.event("file_shared")
def handle_file_shared(event, say, logger, client):    
    # 이미지 파일인지 확인합니다
    file_id = event["file_id"]
    logger.info(f"새로운 이미지가 공유되었습니다. 파일 ID: {file_id}")
    # 파일 정보를 가져옵니다
    file_info = client.files_info(file=file_id)
    file = file_info["file"]
    file_url = file["url_private_download"]
    file_name = current_path / '../image' / file["name"]
    if download_file(file_url, file_name):
        say(f"파일 '{file_name}'을 성공적으로 다운로드했습니다.")
        logger.info(f"파일 '{file_name}'을 성공적으로 다운로드했습니다.")
    else:
        say(f"파일 다운로드에 실패했습니다.")

In [9]:
@app.event("message")
def handle_message_events(event, logger, say):
    # 메시지 텍스트 추출
    message_text = event["text"]
    # 메시지 발신자 ID 추출
    user_id = event["user"]
    # 추출한 정보 사용 예시
    say(f"<@{user_id}>님이 '{message_text}' 메시지를 보냈습니다.")
    logger.info(f"메시지 수신: {message_text}")

In [10]:
# 에러 핸들러
@app.error
def custom_error_handler(error, body, logger):
    print(f"에러 발생: {error}")

In [ ]:
handler = SocketModeHandler(app, SLACK_APP_TOKEN)
handler.start()

INFO:2369379098.py:handle_file_shared:새로운 이미지가 공유되었습니다. 파일 ID: F088RP2G1JR


이벤트 수신: file_shared
이벤트 수신: message


INFO:2369379098.py:handle_message_events:메시지 수신: 
INFO:2369379098.py:handle_file_shared:파일 '/Users/kionkim/Documents/projects/learnus/notebook/../image/hyundai-insurance.jpg'을 성공적으로 다운로드했습니다.
INFO:2369379098.py:handle_file_shared:새로운 이미지가 공유되었습니다. 파일 ID: F088RP2G1JR


이벤트 수신: file_shared


INFO:2369379098.py:handle_file_shared:파일 '/Users/kionkim/Documents/projects/learnus/notebook/../image/hyundai-insurance.jpg'을 성공적으로 다운로드했습니다.
INFO:2369379098.py:handle_file_shared:새로운 이미지가 공유되었습니다. 파일 ID: F088RP2G1JR


이벤트 수신: file_shared


INFO:2369379098.py:handle_file_shared:파일 '/Users/kionkim/Documents/projects/learnus/notebook/../image/hyundai-insurance.jpg'을 성공적으로 다운로드했습니다.


#### Channel ID 파악하기

In [12]:
@app.event("message")
def handle_message_events(event, logger, say):
    # 메시지 텍스트 추출
    message_text = event["text"]
    # 메시지 발신자 ID 추출
    user_id = event["user"]
    # 채널 ID 추출
    channel_id = event["channel"]
    logger.info(f"channel_id: {channel_id}")
    # 추출한 정보 사용 예시
    say(f"<@{user_id}>님이 '{message_text}' 메시지를 보냈습니다.")
    logger.info(f"메시지 수신: {message_text}")

In [ ]:
handler = SocketModeHandler(app, SLACK_APP_TOKEN)
handler.start()